In [512]:
from pandas import read_csv , DataFrame , get_dummies , concat 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
imputer = SimpleImputer()
imputer_mf = SimpleImputer(strategy='most_frequent')
scaler = StandardScaler()

# Preparing Train file.

In [513]:
path_tr = "D:\\datasets\\Titanic\\train.csv"
file_train = read_csv(path_tr)
df_tr = file_train.copy()
df_tr.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
311,312,1,1,"Ryerson, Miss. Emily Borie",female,18.0,2,2,PC 17608,262.3750,B57 B59 B63 B66,C
562,563,0,2,"Norman, Mr. Robert Douglas",male,28.0,0,0,218629,13.5000,NaN,S
536,537,0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.5500,B38,S
264,265,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
196,197,0,3,"Mernagh, Mr. Robert",male,NaN,0,0,368703,7.7500,NaN,Q
396,397,0,3,"Olsson, Miss. Elina",female,31.0,0,0,350407,7.8542,NaN,S
612,613,1,3,"Murphy, Miss. Margaret Jane",female,NaN,1,0,367230,15.5000,NaN,Q
335,336,0,3,"Denkoff, Mr. Mitto",male,NaN,0,0,349225,7.8958,NaN,S
617,618,0,3,"Lobb, Mrs. William Arthur (Cordelia K Stanlick)",female,26.0,1,0,A/5. 3336,16.1000,NaN,S
360,361,0,3,"Skoog, Mr. Wilhelm",male,40.0,1,4,347088,27.9000,NaN,S


In [514]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [515]:
label = df_tr.Survived.to_numpy()
embarked = df_tr.Embarked.to_numpy().reshape(-1,1)
age = df_tr.Age.to_numpy().reshape(-1,1)
sex = df_tr.Sex
dftr = df_tr.drop(['PassengerId','Survived','Name','Ticket','Cabin','Sex','Age','Embarked'],axis=1)

In [516]:
imputed_age = DataFrame(imputer.fit_transform(age),columns=['Age'])
imputed_embarked =DataFrame(imputer_mf.fit_transform(embarked),columns=['Embarked'])
sex_embarked = concat([sex,imputed_embarked],axis=1)
enoded_df = get_dummies(sex_embarked)

In [517]:
ready_df =  concat([dftr,enoded_df],axis=1)

In [518]:
ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   SibSp       891 non-null    int64  
 2   Parch       891 non-null    int64  
 3   Fare        891 non-null    float64
 4   Sex_female  891 non-null    uint8  
 5   Sex_male    891 non-null    uint8  
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
dtypes: float64(1), int64(3), uint8(5)
memory usage: 32.3 KB


In [519]:
final_df = normalize(ready_df)

# Preparing test file.

In [520]:
path_ts = "D:\\datasets\\Titanic\\test.csv"
path_sub = "D:\\datasets\\Titanic\\gender_submission.csv"
file_test = read_csv(path_ts)
file_sub = read_csv(path_sub)
df_ts = file_test.copy()
df_sub = file_sub.copy()
df_ts.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
253,1145,3,"Salander, Mr. Karl Johan",male,24.0,0,0,7266,9.3250,NaN,S
299,1191,3,"Johansson, Mr. Nils",male,29.0,0,0,347467,7.8542,NaN,S
412,1304,3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NaN,S
70,962,3,"Mulvihill, Miss. Bertha E",female,24.0,0,0,382653,7.7500,NaN,Q
118,1010,1,"Beattie, Mr. Thomson",male,36.0,0,0,13050,75.2417,C6,C
296,1188,2,"Laroche, Miss. Louise",female,1.0,1,2,SC/Paris 2123,41.5792,NaN,C
11,903,1,"Jones, Mr. Charles Cresson",male,46.0,0,0,694,26.0000,NaN,S
220,1112,2,"Duran y More, Miss. Florentina",female,30.0,1,0,SC/PARIS 2148,13.8583,NaN,C
223,1115,3,"Karlsson, Mr. Einar Gervasius",male,21.0,0,0,350053,7.7958,NaN,S
136,1028,3,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C


In [521]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [522]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [523]:
label_ts = df_sub.Survived.to_numpy()
embarked_ts = df_ts.Embarked.to_numpy().reshape(-1,1)
age_ts = df_ts.Age.to_numpy().reshape(-1,1)
sex_ts = df_ts.Sex
fare = df_ts.Fare.to_numpy().reshape(-1,1)
dfts = df_ts.drop(['PassengerId','Name','Ticket','Cabin','Sex','Age','Embarked','Fare'],axis=1)

In [524]:
imputed_age_ts = DataFrame(imputer.fit_transform(age_ts),columns=['Age'])
imputed_fare = DataFrame(imputer.fit_transform(fare),columns=['Fare'])
imputed_embarked_ts =DataFrame(imputer_mf.fit_transform(embarked_ts),columns=['Embarked'])
sex_embarked_ts = concat([sex_ts,imputed_embarked_ts],axis=1)
enoded_dfts = get_dummies(sex_embarked_ts)

In [525]:
ready_dfts =  concat([dfts,imputed_fare,enoded_dfts],axis=1)

In [526]:
ready_dfts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   SibSp       418 non-null    int64  
 2   Parch       418 non-null    int64  
 3   Fare        418 non-null    float64
 4   Sex_female  418 non-null    uint8  
 5   Sex_male    418 non-null    uint8  
 6   Embarked_C  418 non-null    uint8  
 7   Embarked_Q  418 non-null    uint8  
 8   Embarked_S  418 non-null    uint8  
dtypes: float64(1), int64(3), uint8(5)
memory usage: 15.2 KB


In [527]:
final_dfts = scaler.fit_transform(ready_dfts)

# Making model.

In [528]:
model = XGBClassifier()
model.fit(final_df,label)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [529]:
model.score(final_df,label)*100

92.25589225589226

In [530]:
model.score(final_dfts,label_ts)*100

94.49760765550239

In [531]:
preds = model.predict(final_dfts)

In [532]:
temp = list()
for i in range(len(df_sub.PassengerId)) :
    temp.append([df_sub.PassengerId.to_numpy()[i],preds[i]])

In [533]:
res_df = DataFrame(temp,columns=['PassengerId','Survived'])
res_df.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [534]:
res_df.to_csv('D:\\datasets\\Titanic\\result.csv',index=False)